<a href="https://colab.research.google.com/github/Champei/mine/blob/main/chapter_appendix-tools-for-deep-learning/jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/"Colab Notebooks"



/content/drive/MyDrive/Colab Notebooks


In [4]:
!pip install jupyter
!jupyter nbconvert --to script "RAG.ipynb"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.0 MB/s eta 0:00:00
[NbConvertApp] Converting notebook RAG.ipynb to script
[NbConvertApp] Writing 17789 bytes to RAG.txt


In [6]:
!ls



 audio_classification.ipynb  'Copy of jupyter.ipynb'	  Untitled
'Copy of audo.ipynb'	     'Copy of MNIST.ipynb'	  Untitled0.ipynb
'Copy of jupyter (1).ipynb'  'Copy of Welcome to Colab'  'Untitled (1)'
'Copy of jupyter (2).ipynb'   RAG.ipynb			  Untitled1.ipynb
'Copy of jupyter (3).ipynb'   RAG.txt


In [7]:
with open("rag_chatbot.py", "w") as f:
    f.write('''<PASTE THE FULL CODE FROM YOUR NOTEBOOK HERE>''')


In [8]:
!ls


 audio_classification.ipynb  'Copy of jupyter.ipynb'	  RAG.txt
'Copy of audo.ipynb'	     'Copy of MNIST.ipynb'	  Untitled
'Copy of jupyter (1).ipynb'  'Copy of Welcome to Colab'   Untitled0.ipynb
'Copy of jupyter (2).ipynb'   rag_chatbot.py		 'Untitled (1)'
'Copy of jupyter (3).ipynb'   RAG.ipynb			  Untitled1.ipynb


In [9]:
%%writefile app.py
import streamlit as st
from rag_chatbot import DocumentStore, Generator, MemoryManager, RAGPipeline, ingest_folder_texts

st.set_page_config(page_title="RAG Chatbot", layout="wide")
st.title("💬 Retrieval-Augmented Chatbot")

st.sidebar.header("⚙️ Settings")
mode = st.sidebar.radio("Generator Mode", ["openai", "local"])
top_k = st.sidebar.slider("Top-K retrieved chunks", 2, 10, 4)
max_turns = st.sidebar.slider("Memory turns", 2, 10, 6)

@st.cache_resource
def init_pipeline(mode, top_k, max_turns):
    store = DocumentStore(persist_path="./rag_store")
    ingest_folder_texts("corpus", store)
    gen = Generator(mode=mode)
    mem = MemoryManager(max_turns=max_turns)
    return RAGPipeline(store=store, generator=gen, memory=mem, top_k=top_k)

rag = init_pipeline(mode, top_k, max_turns)

if "chat_history" not in st.session_state:
    st.session_state["chat_history"] = []

for turn in st.session_state["chat_history"]:
    with st.chat_message(turn["role"]):
        st.markdown(turn["content"])

user_query = st.chat_input("Ask a question...")

if user_query:
    st.session_state["chat_history"].append({"role": "user", "content": user_query})
    with st.chat_message("user"):
        st.markdown(user_query)
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            response = rag.answer(user_query)
        answer = response.get("clarification") if response.get("clarify") else response["answer"]
        st.markdown(answer)
        if not response.get("clarify") and response.get("citations"):
            with st.expander(" Sources and Citations"):
                for c in response["citations"]:
                    st.markdown(f"**[{c['rank']}]** {c['source_id']} — *score: {c['score']:.4f}*")
                    st.caption(c["snippet"])
    st.session_state["chat_history"].append({"role": "assistant", "content": answer})


Writing app.py


In [10]:
!pip install sentence-transformers faiss-cpu numpy transformers torch openai streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.8 MB/s eta 0:00:00


In [12]:
!ngrok config add-authtoken 34nh4N2eC2z7X5obk1C986VYwQ3_4tRDysfSuBDzb182dtgqx


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [19]:
with open("rag_chatbot.py", "w") as f:
    f.write("""
import os, re, time, pickle, json
from pathlib import Path
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional, Tuple

# Optional: OpenAI for generation
try:
    import openai
    OPENAI_AVAILABLE = True
except Exception:
    OPENAI_AVAILABLE = False

# Optional: local HF model for generation
try:
    from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
    HF_AVAILABLE = True
except Exception:
    HF_AVAILABLE = False

# -----------------------
# Utilities
# -----------------------

def simple_text_clean(text: str) -> str:
    text = text.replace("\\r", " ").replace("\\n", " ").strip()
    text = re.sub(r"\\s+", " ", text)
    return text

def chunk_text(text: str, chunk_size: int = 800, overlap: int = 100) -> List[str]:
    text = simple_text_clean(text)
    if len(text) <= chunk_size:
        return [text]
    chunks = []
    start = 0
    while start < len(text):
        end = min(len(text), start + chunk_size)
        chunk = text[start:end]
        chunks.append(chunk)
        if end == len(text):
            break
        start = max(0, end - overlap)
    return chunks

# -----------------------
# Data classes
# -----------------------

@dataclass
class DocChunk:
    id: str
    text: str
    metadata: Dict[str, Any] = field(default_factory=dict)
    embedding: Optional[np.ndarray] = None

# -----------------------
# Document Store + Indexing
# -----------------------

class DocumentStore:
    def __init__(self, embed_model_name: str = "all-MiniLM-L6-v2",
                 faiss_index_path: Optional[str] = None,
                 persist_path: Optional[str] = "./rag_store"):
        self.embed_model_name = embed_model_name
        self.embed_model = SentenceTransformer(embed_model_name)
        self.dim = self.embed_model.get_sentence_embedding_dimension()
        self.index = None
        self.id_to_meta: Dict[str, Dict[str, Any]] = {}
        self.embeddings = None
        self.ids = []
        self.persist_path = Path(persist_path)
        self.persist_path.mkdir(parents=True, exist_ok=True)
        self.faiss_index_path = faiss_index_path or (self.persist_path / "faiss.index")
        self.meta_path = self.persist_path / "metastore.pkl"
        if self.faiss_index_path.exists() and self.meta_path.exists():
            self._load()

    def _build_faiss(self, embeddings: np.ndarray):
        index = faiss.IndexFlatIP(self.dim)
        faiss.normalize_L2(embeddings)
        index.add(embeddings)
        self.index = index

    def _save(self):
        if self.index is None:
            return
        faiss.write_index(self.index, str(self.faiss_index_path))
        with open(self.meta_path, "wb") as f:
            pickle.dump({"ids": self.ids, "id_to_meta": self.id_to_meta}, f)

    def _load(self):
        idx = faiss.read_index(str(self.faiss_index_path))
        self.index = idx
        with open(self.meta_path, "rb") as f:
            d = pickle.load(f)
            self.ids = d["ids"]
            self.id_to_meta = d["id_to_meta"]

    def add_documents(self, docs: List[Tuple[str, str, Dict[str, Any]]],
                      chunk_size: int = 800, chunk_overlap: int = 100):
        chunks: List[DocChunk] = []
        for doc_id, raw_text, meta in docs:
            for i, c in enumerate(chunk_text(raw_text, chunk_size, chunk_overlap)):
                cid = f"{doc_id}__chunk{i}"
                chunks.append(DocChunk(id=cid, text=c, metadata={**meta, "source_id": doc_id, "chunk_index": i}))

        if not chunks:
            return

        texts = [c.text for c in chunks]
        embeddings = self.embed_model.encode(texts, convert_to_numpy=True, show_progress_bar=True)
        faiss.normalize_L2(embeddings)

        if self.index is None:
            self.index = faiss.IndexFlatIP(self.dim)
            self.index.add(embeddings)
            self.ids = [c.id for c in chunks]
            for c in chunks:
                self.id_to_meta[c.id] = {"text": c.text, **c.metadata}
        else:
            self.index.add(embeddings)
            self.ids.extend([c.id for c in chunks])
            for c in chunks:
                self.id_to_meta[c.id] = {"text": c.text, **c.metadata}
        self._save()

    def search(self, query: str, top_k: int = 5) -> List[Dict[str, Any]]:
        q_emb = self.embed_model.encode([query], convert_to_numpy=True)
        faiss.normalize_L2(q_emb)
        if self.index is None or self.index.ntotal == 0:
            return []
        D, I = self.index.search(q_emb, top_k)
        results = []
        for score, idx in zip(D[0], I[0]):
            if idx < 0 or idx >= len(self.ids):
                continue
            doc_id = self.ids[idx]
            meta = self.id_to_meta.get(doc_id, {})
            results.append({
                "id": doc_id,
                "score": float(score),
                "text": meta.get("text", ""),
                "metadata": {k: v for k, v in meta.items() if k != "text"}
            })
        return results

# -----------------------
# Memory Manager
# -----------------------

class MemoryManager:
    def __init__(self, max_turns: int = 6):
        self.max_turns = max_turns
        self.turns: List[Dict[str, Any]] = []

    def add_turn(self, role: str, text: str, retrieved_docs: Optional[List[Dict]] = None):
        import time
        self.turns.append({"role": role, "text": text, "time": time.time(), "retrieved_docs": retrieved_docs or []})
        if len(self.turns) > self.max_turns:
            self.turns = self.turns[-self.max_turns:]

    def get_memory_prompt(self) -> str:
        if not self.turns:
            return ""
        return "\\n".join([f"{t['role']}: {t['text']}" for t in self.turns])

# -----------------------
# Generator
# -----------------------

class Generator:
    def __init__(self, mode: str = "openai", openai_model: str = "gpt-4o-mini", hf_model_name: str = "gpt2"):
        self.mode = mode
        self.openai_model = openai_model
        self.hf_model_name = hf_model_name
        self.hf_pipe = None
        if mode == "openai" and OPENAI_AVAILABLE:
            openai.api_key = os.environ.get("OPENAI_API_KEY", "")
        if mode == "local" and HF_AVAILABLE:
            try:
                self.hf_pipe = pipeline("text-generation", model=self.hf_model_name, device_map="auto")
            except Exception as e:
                print("Failed to load HF model:", e)

    def generate(self, prompt: str, max_tokens: int = 512, temperature: float = 0.0) -> str:
        if self.mode == "openai" and OPENAI_AVAILABLE:
            resp = openai.ChatCompletion.create(
                model=self.openai_model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=max_tokens,
                temperature=temperature,
                n=1
            )
            return resp.choices[0].message.content.strip()
        elif self.mode == "local" and self.hf_pipe is not None:
            out = self.hf_pipe(prompt, max_length=max_tokens + len(prompt.split()), do_sample=False)
            return out[0]["generated_text"][len(prompt):].strip()
        else:
            return "⚠️ No generator configured — set OPENAI_API_KEY or use HF model."

# -----------------------
# RAG Pipeline
# -----------------------

class RAGPipeline:
    def __init__(self, store: DocumentStore, generator: Generator, memory: Optional[MemoryManager] = None, top_k: int = 4):
        self.store = store
        self.generator = generator
        self.memory = memory or MemoryManager()
        self.top_k = top_k

    def _build_context_block(self, retrieved: List[Dict[str, Any]]) -> str:
        parts = []
        for i, r in enumerate(retrieved, start=1):
            source = r["metadata"].get("source_id", r["id"])
            snippet = r["text"][:600].strip()
            parts.append(f"[{i}] Source: {source} | Score: {r['score']:.4f}\\nSnippet: {snippet}")
        return "\\n\\n".join(parts)

    def _build_prompt(self, user_query: str, retrieved: List[Dict[str, Any]]) -> str:
        memory_block = self.memory.get_memory_prompt()
        context_block = self._build_context_block(retrieved)
        prompt = "You are a helpful assistant that answers using ONLY the provided context.\\n"
        prompt += "Cite sources inline like [1], [2], etc. Include a 'Sources' list at the end.\\n\\n"
        if memory_block:
            prompt += f"Conversation memory:\\n{memory_block}\\n\\n"
        if context_block:
            prompt += f"Context:\\n{context_block}\\n\\n"
        prompt += f"User question:\\n{user_query}\\n\\n"
        return prompt

    def answer(self, user_query: str, k: Optional[int] = None) -> Dict[str, Any]:
        k = k or self.top_k
        if len(user_query.strip().split()) <= 2:
            clar_prompt = f"User asked: '{user_query}'. Suggest 1-2 clarifying questions."
            clar_question = self.generator.generate(clar_prompt, max_tokens=60, temperature=0.7)
            self.memory.add_turn("assistant", clar_question, [])
            return {"clarify": True, "clarification": clar_question, "sources": []}
        retrieved = self.store.search(user_query, top_k=k)
        self.memory.add_turn("user", user_query, [r["id"] for r in retrieved])
        prompt = self._build_prompt(user_query, retrieved)
        gen_text = self.generator.generate(prompt, max_tokens=512)
        self.memory.add_turn("assistant", gen_text, [r["id"] for r in retrieved])
        citations = [{"rank": i, "id": r["id"], "source_id": r["metadata"].get("source_id"), "score": r["score"], "snippet": r["text"][:300]} for i, r in enumerate(retrieved, start=1)]
        return {"clarify": False, "answer": gen_text, "citations": citations, "retrieved_count": len(retrieved)}

def load_text_file(path: str) -> str:
    return Path(path).read_text(encoding="utf8")

def ingest_folder_texts(folder: str, store: DocumentStore):
    folder = Path(folder)
    docs = []
    for p in folder.glob("**/*"):
        if p.is_file() and p.suffix.lower() in [".txt", ".md"]:
            docs.append((p.stem, load_text_file(str(p)), {"path": str(p)}))
    store.add_documents(docs)

if __name__ == "__main__":
    print(" RAG Chatbot module loaded. Use quick_demo_local() or Streamlit interface to test.")
""")


In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(8501)
print(" Open this link:", public_url)

!streamlit run app.py --server.port 8501 --server.enableCORS false --server.enableXsrfProtection false


 Open this link: NgrokTunnel: "https://joana-pneumatophorous-semblably.ngrok-free.dev" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.169.180.13:8501

2025-10-30 20:29:12.240785: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761856152.283528    8281 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761856152.297547    8281 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761856152.337168    8281 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the 